In [1]:
import requests, json
import pandas as pd
from pandas_profiling import ProfileReport
from pprint import pprint
from bs4 import BeautifulSoup as bs
import datetime
import pickle

In [2]:
def get_vacancies(page=0):
    return requests.get(
                        'https://api.hh.ru/vacancies'
                        '?area=113&area=40&area=28&area=1001'
                        '&text=python'
                        '&search_field=name&search_field=description'
                        '&employment=full&employment=part'
                        '&schedule=remote'
                        '&professional_role=96'
                        '&industry=7&industry=8&industry=9&industry=11'
                        '&industry=39.448&industry=39.444&industry=39.445'
                        '&industry=39.446&industry=41.525'
                        '&industry=43&industry=44.391&industry=44.396'
                        '&industry=44.392&industry=44.393&industry=49.404'
                        '&industry=49.411&industry=49.412&industry=389'
                        '&date_from=2022-09-01'
                        f'&page={page}'
                        '&per_page=100'
                        )

In [3]:
vacancies = get_vacancies()

In [4]:
def get_df(items):
    df = pd.DataFrame.from_records(items)
    df.drop(columns=[
                    'has_test',
                    'premium',
                    'address',
                    'department',
                    'response_letter_required',
                    'sort_point_distance',
                    'response_url',
                    'relations',
                    'schedule',
                    'working_days',
                    'working_time_intervals',
                    'working_time_modes',
                    'accept_temporary',
                    'apply_alternate_url',
                    'insider_interview',
                    'adv_response_url',
                    'contacts',    
                ], inplace=True)
    df['area'] = df['area'].apply(lambda x: x.get('name'))
    df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
    df['type'] = df['type'].apply(lambda x: x.get('name'))
    df['employer'] = df['employer'].apply(lambda x: x.get('name'))
    df = pd.concat([df.drop(['snippet'], axis=1), df['snippet'].apply(pd.Series)], axis=1)
    return df

In [5]:
df = pd.DataFrame()
for page in range(vacancies.json()['pages']):
    v = get_vacancies(page=page)
    d = get_df(v.json()['items'])
    df = df.append(d, ignore_index=True)

/tmp/ipykernel_605154/980802284.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
/tmp/ipykernel_605154/980802284.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
/tmp/ipykernel_605154/980802284.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
/tmp/ipykernel_605154/980802284.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future 

In [6]:
deep_vac = []
for i in df.itertuples():
    if i.url:
        r = requests.get(i.url)
        deep_vac.append(r.json())

In [7]:
deep_df = pd.DataFrame.from_records(deep_vac)

In [8]:
def get_deep_df(df):
    df = df.drop(columns=[
        'premium',
        'billing_type',
        'relations',
        'accept_handicapped',
        'insider_interview',
        'response_letter_required',
        'type',
        'address',
        'allow_messages',
        'schedule',
        'department',
        'contacts',
        'contacts',
        'branded_description',
        'vacancy_constructor_template',
        'accept_kids',
        'archived',
        'response_url',
        'code',
        'quick_responses_allowed',
        'driver_license_types',
        'accept_incomplete_resumes',
        'initial_created_at',
        'negotiations_url',
        'suitable_resumes_url',
        'has_test',
        'test',
        'working_days',
        'working_time_intervals',
        'working_time_modes',
        'accept_temporary',
        'hidden',
                ])
    
    for name in ['area', 'employer', 'experience', 'employment']:
        df[name] = df[name].apply(lambda x: x.get('name'))
    df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
    
    def skills_to_list(x):
        if x:
            if isinstance(x[0], dict):
                items = [i['name'] for i in x]
                return items
            else:
                return x

    for name in ['key_skills', 'specializations', 'professional_roles']:
        df[name] = df[name].apply(skills_to_list)
        
    df['languages'] = df['languages'].apply(lambda x: x if x else None)
    
    def get_text(x):
        if x:
            soup = bs(x, features="html.parser")
            text = soup.get_text()
            return text
        
    df['description'] = df['description'].apply(get_text)
    
    return df

In [9]:
deep_df1 = get_deep_df(deep_df)

/tmp/ipykernel_605154/1820415902.py:39: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
/tmp/ipykernel_605154/1820415902.py:39: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
/tmp/ipykernel_605154/1820415902.py:39: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
/tmp/ipykernel_605154/1820415902.py:39: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a fut

In [10]:
filtered_skills = [
 'A/B тесты',
 'API',
 'AirFlow',
 'Airflow',
 'Alembic',
 'Ansible',
 'Apache',
 'Apache Airflow',
 'Apache Kafka',
 'Asyncio',
 'Big Data',
 'BigData',
 'CI/CD',
 'Celery',
 'ClickHouse',
 'Clickhouse',
 'Computer Vision',
 'Computer vision',
 'Data Analysis',
 'Data Mining',
 'Data Science',
 'Data engineer',
 'Data enineer',
 'Data science',
 'Databases',
 'DevOps',
 'DevSecOps',
 'Development',
 'Django',
 'Django 2',
 'Django Framework',
 'Django REST',
 'Django Rest Framework',
 'Docker',
 'Docker-compose',
 'ElascticSearch',
 'ElastickSearch',
 'Elasticsearch',
 'Fast API',
 'FastAPI',
 'FastApi',
 'Flask',
 'GIL',
 'Git',
 'Git flow',
 'GitHub',
 'GitLab CI',
 'Gitlab',
 'Gitlab CI',
 'Grafana',
 'GraphQL',
 'Graphana',
 'Graphite',
 'Graphql',
 'HTTP',
 'Hadoop',
 'Helm',
 'JSON API',
 'Jenkins',
 'Jira',
 'Kafka',
 'Keras',
 'Kibana',
 'Kubeflow',
 'Kubernetes',
 'Kuberntes',
 'Logging',
 'ML',
 'MLOp',
 'MLOps',
 'Machine Learning',
 'Mathematical Statistics',
 'Memcached',
 'Microservices',
 'MongoDB',
 'Multithread Programming',
 'Multithreading',
 'MySQL',
 'Nginx',
 'NoSQL',
 'NumPy',
 'Numpy',
 'OLAP',
 'OOP',
 'ORM',
 'Olap (online analytical processing)',
 'OpenAPI',
 'OpenCV',
 'Pandas',
 'Pl/Python',
 'Plotly',
 'PostgreSQL',
 'Postgres',
 'Prometheus',
 'PyCharm',
 'PyTorch',
 'Pytest',
 'Python',
 'Python 3',
 'Pytorch',
 'QA',
 'REST',
 'REST API',
 'RESTful API',
 'RabbitMQ',
 'Redis',
 'Rest Api',
 'SQL',
 'SQLAlchemy',
 'SciPy',
 'Selenium',
 'Sklearn',
 'Spark',
 'Statistics',
 'Swagger',
 'TCP/IP',
 'TDD',
 'TensorFlow',
 'Tensorflow',
 'Terraform',
 'Tornado',
 'Variational Autoencoders',
 'Web API',
 'aiohttp',
 'airflow',
 'ansible',
 'asyncio',
 'bs4',
 'clickhouse',
 'docker',
 'fastapi',
 'hadoop',
 'image processing',
 'iohttp',
 'keras',
 'kubernetes',
 'machine learning',
 'neo4j',
 'ngnix',
 'pytest',
 'python, aiohttp, fastapi, trending, linux',
 'pytorch',
 'requests',
 'sklearn',
 'Машинное обучение',
 'Многопоточное программирование',
 'Многопоточность',
 'Парсинг',
 'знание web-фреймворков в python (любых)',
]
stay = []
for i in deep_df1.itertuples():
    if i.key_skills:
        for j in i.key_skills:
            if j in filtered_skills:
                stay.append(i.id)
                break

In [11]:
deep_df2 = deep_df1[deep_df1['id'].isin(stay)]

In [12]:
stop_words = [
 '(Angular)',
 '(C#)',
 '(C++)',
 '(Hotcore)',
 '(Laravel)',
 '(Lua)',
 '(NixOS)',
 '(Oracle)',
 '(React',
 '(Ruby',
 '.NET',
 '1С',
 'C',
 'C#',
 'C#/.NET',
 'C++',
 'C++/Старший',
 'C/Go',
 'DilOS',
 'Fraud',
 'Frontend',
 'Frontend-разработчик',
 'Full-Stack',
 'Full-stack',
 'Fullstack',
 'Fullstack-разработчик',
 'Go',
 'Go,',
 'Golang',
 'Golang,',
 'Golang-разработчик',
 'HR',
 'Java',
 'Java+Python/Oracle',
 'Java,',
 'JavaScript)',
 'JavaScript,',
 'Javascript',
 'Next.js)',
 'Node',
 'Node.JS-developer',
 'Node.js',
 'Oracle/Data',
 'PHP',
 'PHP-программист',
 'PHP-разработчик',
 'Perl-разработчик',
 'Rails)',
 'Rails),',
 'Scala',
 'Unity',
 'full-stack',
 'fullstack',
 'С++',
 'С/С++',
]
drop = []
for i in deep_df2.itertuples():
    if i.name:
        for j in stop_words:
            if j in i.name:
                drop.append(i.id)
                break

In [13]:
deep_df3 = deep_df2.drop(deep_df2[deep_df2['id'].isin(drop)].index.to_list())

In [14]:
deep_df3.sort_values(by=['employer'], inplace=True)

In [15]:
deep_df3.drop(
    ['employment', 'specializations', 'published_at', 
     'created_at', 'apply_alternate_url', 'gross', 'professional_roles'], 
    axis=1, 
    inplace=True
    )

In [16]:
deep_df3.drop(deep_df3[deep_df3['languages'].notnull()].index, inplace=True)
deep_df3.drop(['languages'], axis=1, inplace=True)

In [17]:
deep_df3.sort_values(by=['employer'], inplace=True)

In [18]:
k_skills = set()
for i in deep_df3['key_skills']:
    k_skills.update(list(i))

In [19]:
k_skills

{'.NET Framework',
 'A/B тесты',
 'AIOHTTP',
 'API',
 'ASR',
 'AWS',
 'AWT',
 'AWX',
 'Actions',
 'Agile Project Management',
 'Aiohttp',
 'AirFlow',
 'Airflow',
 'Android',
 'Ansible',
 'Apache',
 'Apache Airflow',
 'Apache Cassandra',
 'Apache Hadoop',
 'Apache Kafka',
 'Apache NiFi',
 'Apache Spark',
 'Application security',
 'Appsflyer',
 'Asyncio',
 'Atlassian Confluence',
 'Atlassian Jira',
 'Azure',
 'B2B',
 'BI',
 'BI-отчетность',
 'Backend',
 'Bash',
 'Big Data',
 'Blending',
 'Bootstrap',
 'Burp Suite',
 'C#',
 'C++',
 'C/C++',
 'CI',
 'CI/CD',
 'CSS',
 'CSS3',
 'CTE',
 'Cassandra',
 'Celery',
 'CentOS',
 'ClickHouse',
 'Clickhouse',
 'DAM',
 'DAST',
 'DRF',
 'DRY',
 'DWH',
 'Dash',
 'Data Engineering',
 'Data Science',
 'Data governance',
 'Databases',
 'Databricks',
 'DevOps',
 'DevSecOps',
 'Django',
 'Django Framework',
 'Django REST',
 'Django REST Framework',
 'Django Rest Framework',
 'Docker',
 'Docker-compose',
 'ELK',
 'ETCD',
 'ETL',
 'Elasticsearch',
 'FAST API',


In [20]:
stop_words = [
    'Английский язык', '1С программирование', '.NET', '.NET Core', '.NET Framework',
    'Adobe Photoshop', 'C#', 'C++', 'C/C++', 'Java', 'JavaScript', 'Lua', 
    'Node.js', 'PHP', 'Perl', 'React', 'React.js', 'SCALA', 'Vue.js', 'С#',
    'С++',
]
for i in deep_df3.itertuples():
    if i.key_skills:
        for j in stop_words:
            if j in i.key_skills:
                drop.append(i.id)
                break
deep_df4 = deep_df3.drop(deep_df3[deep_df3['id'].isin(drop)].index.to_list())

In [21]:
with open("../data/indexes", "rb") as fp:   # Unpickling
    indexes = pickle.load(fp)
print(len(indexes))

1144


In [22]:
drop = indexes.intersection(set(deep_df4.id))
deep_df5 = deep_df4.drop(deep_df4[deep_df4['id'].isin(drop)].index.to_list())

In [23]:
deep_df5.drop_duplicates(subset=['description'], inplace=True)
deep_df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37 entries, 261 to 200
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             37 non-null     object 
 1   name           37 non-null     object 
 2   area           37 non-null     object 
 3   experience     37 non-null     object 
 4   description    37 non-null     object 
 5   key_skills     37 non-null     object 
 6   employer       37 non-null     object 
 7   alternate_url  37 non-null     object 
 8   from           13 non-null     float64
 9   to             12 non-null     float64
 10  currency       17 non-null     object 
dtypes: float64(2), object(9)
memory usage: 3.5+ KB


In [24]:
dt = datetime.datetime.now().strftime("%m-%d-%Y(%H-%M)")
deep_df5.to_csv(f'../data/result_deep_sorted_{dt}.csv', index=False)

In [25]:
indexes.update(deep_df5.id)
with open("../data/indexes", "wb") as fp:   #Pickling
    pickle.dump(indexes, fp)
print(len(indexes))

1181
